In [1]:
import numpy as np
import h5py

import matplotlib.pyplot as plt
%matplotlib widget

In [3]:
data = {}
new_data = {}

with h5py.File('../EOS/LS220_234r_136t_50y_analmu_20091212_SVNr26.h5') as f:
    keys = list(f.keys())
    for key in keys:
        data[key] = np.array(f[key])

In [7]:
ggrav = 6.6738480e-8 #cgs
clite = 2.99792458e10 #cgs
msun = 1.9884e33 #cgs

Clength = ggrav * msun / clite**2

MeVc2_to_grams = 1.78266232885782e-27
baryonic_mass = 939.56542052 # Mev / c^2


In [44]:
%%timeit

VC_baryon_mass_density = 10**data['logrho']

VC_ye_0_idx = np.argmin(data['logenergy'][:, 0, :], axis = 0)
entry = np.array(range(0, len(VC_ye_0_idx)))
VC_eps = 10**data['logenergy'][VC_ye_0_idx, 0, entry] - data['energy_shift'][0] # erg / gr
VC_pressure = 10**data['logpress'][VC_ye_0_idx, 0, entry] # dyn / cm^2
VC_cs2 = data['cs2'][VC_ye_0_idx, 0, entry]

VC_energy_density = baryon_mass_density * (1 + VC_eps/clite**2)

VC_logh              = np.array(cs2) * np.log((energy_density + pressure/clite**2) / baryon_mass_density)
VC_logh[VC_logh < 0]    = 1

54.7 µs ± 862 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [45]:
%%timeit

baryon_mass_density = []
energy_density = []
pressure = []
cs2 = []
ye_0_list = []

for r_idx, r in enumerate(10**data['logrho']):
    baryon_mass_density.append(r)  # gr / cm^3

    ye_0_idx = np.argmin(data['logenergy'][:, 0, r_idx])
    ye_0_list.append(ye_0_idx)
    
    eps = 10**data['logenergy'][ye_0_idx, 0, r_idx] - data['energy_shift'][0] # erg / gr
    pp = 10**data['logpress'][ye_0_idx, 0, r_idx] # dyn / cm^2

    energy_density.append(baryon_mass_density[-1] * (1 + eps/clite**2)) # gr / cm^3
    pressure.append(pp) # dyn / cm^2
    cs2.append(data['cs2'][ye_0_idx, 0, r_idx]) # cm^2 / s^2

baryon_mass_density   =  np.array(baryon_mass_density)
baryon_number_density =  baryon_mass_density / baryonic_mass / MeVc2_to_grams  # 1 / cm^3
energy_density    =  np.array(energy_density)   
pressure          =  np.array(pressure)   
logh              = np.array(cs2) * np.log((energy_density + pressure/clite**2) / baryon_mass_density)
logh[logh < 0]    = 1


650 µs ± 4.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [43]:
# TESTS

# same rho
np.testing.assert_allclose(baryon_mass_density, VC_baryon_mass_density)
print('Passed')
# same ye_0
np.testing.assert_array_equal(ye_0_list, VC_ye_0_idx)
print('Passed')
# same P
np.testing.assert_allclose(pressure, VC_pressure)
print('Passed')
# same e
np.testing.assert_allclose(energy_density, VC_energy_density)
print('Passed')
# same cs2
np.testing.assert_allclose(cs2, VC_cs2)
print('Passed')
# same logh
np.testing.assert_allclose(logh, VC_logh)
print('Passed')


Passed
Passed
Passed
Passed
Passed
Passed


In [8]:
# import os

# os.system('rm EOS_RNSID.LS220')

# with open("EOS_RNSID.LS220", "a") as f:
#     f.write(f"{len(baryon_mass_density)}\n")
#     np.savetxt(f, np.c_[energy_density, pressure, logh, baryon_number_density])
    

In [9]:
rho_central = baryon_number_density[-1] * baryonic_mass * MeVc2_to_grams / (msun / Clength**3)
print(f'{rho_central = }')

rho_central = 0.016188709313274225
